In [1]:
import math
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.backends import cudnn
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms
from torchvision.datasets import CIFAR10
import os
import sys
import time
from wrn import WideResNet

In [2]:
SAVE_PATH = 'WRN28@10_cifar10.pth'
device = 'cuda:1'
start_epoch = 1
num_epochs = 200
batch_size = 128
optim_type = 'SGD'
num_classes = 10
root_dir = '../data/cifar10'
d = 28 #
k = 10

In [3]:
mean = (0.4914, 0.4822, 0.4465)
std = (0.2023, 0.1994, 0.2010)

In [4]:
# Only for cifar-10
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [5]:
def learning_rate(init, epoch):
    optim_factor = 0
    if(epoch > 160):
        optim_factor = 3
    elif(epoch > 120):
        optim_factor = 2
    elif(epoch > 60):
        optim_factor = 1

    return init*math.pow(0.2, optim_factor)

In [6]:
def get_hms(seconds):
    m, s = divmod(seconds, 60)
    h, m = divmod(m, 60)

    return h, m, s


In [7]:
best_acc = 0

## Augmentation

In [8]:
transform_train = transforms.Compose([transforms.RandomCrop(size=32, padding=4),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=mean, std=std),])
transform_test = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize(mean=mean, std=std),])

In [9]:
trainset = CIFAR10(root=root_dir, train=True, download=True, transform=transform_train)
testset = CIFAR10(root=root_dir, train=False, download=True, transform=transform_test)

Files already downloaded and verified
Files already downloaded and verified


In [10]:
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(testset, batch_size=100, shuffle=False)

In [11]:
net = WideResNet(depth=d, widen_factor=k)
file_name = 'wide-resnet-'+str(d)+'x'+str(k)
criterion = nn.CrossEntropyLoss()

| Wide-Resnet 28x10


## Train

In [12]:
for epoch in range(start_epoch, start_epoch+num_epochs):
    net.cuda(device)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    optimizer = optim.SGD(net.parameters(), lr=learning_rate(0.1, epoch), momentum=0.9, weight_decay=5e-4, nesterov=True)
    print('\n=> Training Epoch #%d, LR=%.4f' %(epoch, learning_rate(0.1, epoch)))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.cuda(device), targets.cuda(device)
        optimizer.zero_grad()
        inputs, targets = Variable(inputs), Variable(targets)
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.data.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()
        
        if batch_idx % 25 == 0:
            print('| Epoch [%3d/%3d] Iter [%3d/%3d]\t\tLoss: %.4f Acc@1: %.3f%%' %(epoch, num_epochs, batch_idx+1,
                                                                               (len(trainset)//batch_size)+1, loss.data.item(), 100.*correct/total))
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.cuda(device), targets.cuda(device)
        inputs, targets = Variable(inputs, volatile=True), Variable(targets)
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.data.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()
    
    acc = 100. * correct / total
    print("\n| Validation Epoch #%d\t\t\tLoss: %.4f Acc@1: %.2f%%" %(epoch, loss.data.item(), acc))
    
    if acc > best_acc:
        print('| Saving Best model...\t\t\tTop1 = %.2f%%' %(acc))
        state = {
                'net':net.modules,
                'acc':acc,
                'epoch':epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        save_point = './checkpoint/'
        if not os.path.isdir(save_point):
            os.mkdir(save_point)
        torch.save(state, save_point+file_name+'.t7')
        best_acc = acc


=> Training Epoch #1, LR=0.1000
| Epoch [  1/200] Iter [  1/391]		Loss: 2.3166 Acc@1: 8.000%
| Epoch [  1/200] Iter [ 26/391]		Loss: 1.9582 Acc@1: 21.000%
| Epoch [  1/200] Iter [ 51/391]		Loss: 1.7600 Acc@1: 25.000%
| Epoch [  1/200] Iter [ 76/391]		Loss: 1.5144 Acc@1: 29.000%
| Epoch [  1/200] Iter [101/391]		Loss: 1.6807 Acc@1: 31.000%
| Epoch [  1/200] Iter [126/391]		Loss: 1.2663 Acc@1: 34.000%
| Epoch [  1/200] Iter [151/391]		Loss: 1.3094 Acc@1: 36.000%
| Epoch [  1/200] Iter [176/391]		Loss: 1.3931 Acc@1: 38.000%
| Epoch [  1/200] Iter [201/391]		Loss: 1.3363 Acc@1: 39.000%
| Epoch [  1/200] Iter [226/391]		Loss: 1.1014 Acc@1: 41.000%
| Epoch [  1/200] Iter [251/391]		Loss: 1.2203 Acc@1: 43.000%
| Epoch [  1/200] Iter [276/391]		Loss: 1.0440 Acc@1: 44.000%
| Epoch [  1/200] Iter [301/391]		Loss: 1.2405 Acc@1: 45.000%
| Epoch [  1/200] Iter [326/391]		Loss: 1.0841 Acc@1: 46.000%
| Epoch [  1/200] Iter [351/391]		Loss: 1.1966 Acc@1: 47.000%
| Epoch [  1/200] Iter [376/391]		Loss

/home/skkulab/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.



| Validation Epoch #1			Loss: 1.9873 Acc@1: 55.00%
| Saving Best model...			Top1 = 55.00%

=> Training Epoch #2, LR=0.1000
| Epoch [  2/200] Iter [  1/391]		Loss: 0.8834 Acc@1: 64.000%
| Epoch [  2/200] Iter [ 26/391]		Loss: 1.0821 Acc@1: 64.000%
| Epoch [  2/200] Iter [ 51/391]		Loss: 0.8829 Acc@1: 64.000%
| Epoch [  2/200] Iter [ 76/391]		Loss: 0.9673 Acc@1: 64.000%
| Epoch [  2/200] Iter [101/391]		Loss: 0.9550 Acc@1: 65.000%
| Epoch [  2/200] Iter [126/391]		Loss: 0.8953 Acc@1: 65.000%
| Epoch [  2/200] Iter [151/391]		Loss: 0.9676 Acc@1: 65.000%
| Epoch [  2/200] Iter [176/391]		Loss: 0.9150 Acc@1: 65.000%
| Epoch [  2/200] Iter [201/391]		Loss: 0.9251 Acc@1: 65.000%
| Epoch [  2/200] Iter [226/391]		Loss: 0.9824 Acc@1: 65.000%
| Epoch [  2/200] Iter [251/391]		Loss: 0.8517 Acc@1: 66.000%
| Epoch [  2/200] Iter [276/391]		Loss: 0.8937 Acc@1: 66.000%
| Epoch [  2/200] Iter [301/391]		Loss: 0.7505 Acc@1: 66.000%
| Epoch [  2/200] Iter [326/391]		Loss: 0.9191 Acc@1: 66.000%
| Epoch 

| Epoch [  9/200] Iter [151/391]		Loss: 0.5255 Acc@1: 84.000%
| Epoch [  9/200] Iter [176/391]		Loss: 0.5556 Acc@1: 84.000%
| Epoch [  9/200] Iter [201/391]		Loss: 0.5049 Acc@1: 84.000%
| Epoch [  9/200] Iter [226/391]		Loss: 0.4705 Acc@1: 84.000%
| Epoch [  9/200] Iter [251/391]		Loss: 0.4567 Acc@1: 84.000%
| Epoch [  9/200] Iter [276/391]		Loss: 0.5377 Acc@1: 84.000%
| Epoch [  9/200] Iter [301/391]		Loss: 0.5712 Acc@1: 84.000%
| Epoch [  9/200] Iter [326/391]		Loss: 0.4975 Acc@1: 84.000%
| Epoch [  9/200] Iter [351/391]		Loss: 0.5223 Acc@1: 84.000%
| Epoch [  9/200] Iter [376/391]		Loss: 0.3130 Acc@1: 84.000%

| Validation Epoch #9			Loss: 0.8416 Acc@1: 77.00%

=> Training Epoch #10, LR=0.1000
| Epoch [ 10/200] Iter [  1/391]		Loss: 0.4047 Acc@1: 85.000%
| Epoch [ 10/200] Iter [ 26/391]		Loss: 0.3140 Acc@1: 87.000%
| Epoch [ 10/200] Iter [ 51/391]		Loss: 0.6204 Acc@1: 85.000%
| Epoch [ 10/200] Iter [ 76/391]		Loss: 0.4065 Acc@1: 85.000%
| Epoch [ 10/200] Iter [101/391]		Loss: 0.3419


| Validation Epoch #16			Loss: 0.6309 Acc@1: 78.00%

=> Training Epoch #17, LR=0.1000
| Epoch [ 17/200] Iter [  1/391]		Loss: 0.3848 Acc@1: 85.000%
| Epoch [ 17/200] Iter [ 26/391]		Loss: 0.2369 Acc@1: 88.000%
| Epoch [ 17/200] Iter [ 51/391]		Loss: 0.2857 Acc@1: 87.000%
| Epoch [ 17/200] Iter [ 76/391]		Loss: 0.3571 Acc@1: 87.000%
| Epoch [ 17/200] Iter [101/391]		Loss: 0.3434 Acc@1: 86.000%
| Epoch [ 17/200] Iter [126/391]		Loss: 0.4298 Acc@1: 87.000%
| Epoch [ 17/200] Iter [151/391]		Loss: 0.4789 Acc@1: 86.000%
| Epoch [ 17/200] Iter [176/391]		Loss: 0.3867 Acc@1: 86.000%
| Epoch [ 17/200] Iter [201/391]		Loss: 0.3064 Acc@1: 87.000%
| Epoch [ 17/200] Iter [226/391]		Loss: 0.3709 Acc@1: 87.000%
| Epoch [ 17/200] Iter [251/391]		Loss: 0.3327 Acc@1: 87.000%
| Epoch [ 17/200] Iter [276/391]		Loss: 0.4379 Acc@1: 87.000%
| Epoch [ 17/200] Iter [301/391]		Loss: 0.4029 Acc@1: 87.000%
| Epoch [ 17/200] Iter [326/391]		Loss: 0.2755 Acc@1: 86.000%
| Epoch [ 17/200] Iter [351/391]		Loss: 0.405

...


| Validation Epoch #168			Loss: 0.1722 Acc@1: 94.00%

=> Training Epoch #169, LR=0.0008
| Epoch [169/200] Iter [  1/391]		Loss: 0.0021 Acc@1: 100.000%
| Epoch [169/200] Iter [ 26/391]		Loss: 0.0023 Acc@1: 100.000%
| Epoch [169/200] Iter [ 51/391]		Loss: 0.0021 Acc@1: 100.000%
| Epoch [169/200] Iter [ 76/391]		Loss: 0.0013 Acc@1: 100.000%
| Epoch [169/200] Iter [101/391]		Loss: 0.0051 Acc@1: 100.000%
| Epoch [169/200] Iter [126/391]		Loss: 0.0012 Acc@1: 100.000%
| Epoch [169/200] Iter [151/391]		Loss: 0.0028 Acc@1: 99.000%
| Epoch [169/200] Iter [176/391]		Loss: 0.0012 Acc@1: 99.000%
| Epoch [169/200] Iter [201/391]		Loss: 0.0012 Acc@1: 99.000%
| Epoch [169/200] Iter [226/391]		Loss: 0.0011 Acc@1: 99.000%
| Epoch [169/200] Iter [251/391]		Loss: 0.0025 Acc@1: 99.000%
| Epoch [169/200] Iter [276/391]		Loss: 0.0021 Acc@1: 99.000%
| Epoch [169/200] Iter [301/391]		Loss: 0.0030 Acc@1: 99.000%
| Epoch [169/200] Iter [326/391]		Loss: 0.0015 Acc@1: 99.000%
| Epoch [169/200] Iter [351/391]		Los

| Epoch [176/200] Iter [226/391]		Loss: 0.0017 Acc@1: 99.000%
| Epoch [176/200] Iter [251/391]		Loss: 0.0016 Acc@1: 99.000%
| Epoch [176/200] Iter [276/391]		Loss: 0.0017 Acc@1: 99.000%
| Epoch [176/200] Iter [301/391]		Loss: 0.0018 Acc@1: 99.000%
| Epoch [176/200] Iter [326/391]		Loss: 0.0027 Acc@1: 99.000%
| Epoch [176/200] Iter [351/391]		Loss: 0.0017 Acc@1: 99.000%
| Epoch [176/200] Iter [376/391]		Loss: 0.0017 Acc@1: 99.000%

| Validation Epoch #176			Loss: 0.1517 Acc@1: 95.00%

=> Training Epoch #177, LR=0.0008
| Epoch [177/200] Iter [  1/391]		Loss: 0.0019 Acc@1: 100.000%
| Epoch [177/200] Iter [ 26/391]		Loss: 0.0024 Acc@1: 99.000%
| Epoch [177/200] Iter [ 51/391]		Loss: 0.0014 Acc@1: 99.000%
| Epoch [177/200] Iter [ 76/391]		Loss: 0.0022 Acc@1: 99.000%
| Epoch [177/200] Iter [101/391]		Loss: 0.0017 Acc@1: 99.000%
| Epoch [177/200] Iter [126/391]		Loss: 0.0020 Acc@1: 99.000%
| Epoch [177/200] Iter [151/391]		Loss: 0.0054 Acc@1: 99.000%
| Epoch [177/200] Iter [176/391]		Loss: 0.

| Epoch [184/200] Iter [ 51/391]		Loss: 0.0020 Acc@1: 99.000%
| Epoch [184/200] Iter [ 76/391]		Loss: 0.0013 Acc@1: 99.000%
| Epoch [184/200] Iter [101/391]		Loss: 0.0024 Acc@1: 99.000%
| Epoch [184/200] Iter [126/391]		Loss: 0.0018 Acc@1: 99.000%
| Epoch [184/200] Iter [151/391]		Loss: 0.0028 Acc@1: 99.000%
| Epoch [184/200] Iter [176/391]		Loss: 0.0020 Acc@1: 99.000%
| Epoch [184/200] Iter [201/391]		Loss: 0.0017 Acc@1: 99.000%
| Epoch [184/200] Iter [226/391]		Loss: 0.0015 Acc@1: 99.000%
| Epoch [184/200] Iter [251/391]		Loss: 0.0023 Acc@1: 99.000%
| Epoch [184/200] Iter [276/391]		Loss: 0.0012 Acc@1: 99.000%
| Epoch [184/200] Iter [301/391]		Loss: 0.0029 Acc@1: 99.000%
| Epoch [184/200] Iter [326/391]		Loss: 0.0029 Acc@1: 99.000%
| Epoch [184/200] Iter [351/391]		Loss: 0.0019 Acc@1: 99.000%
| Epoch [184/200] Iter [376/391]		Loss: 0.0031 Acc@1: 99.000%

| Validation Epoch #184			Loss: 0.1575 Acc@1: 95.00%

=> Training Epoch #185, LR=0.0008
| Epoch [185/200] Iter [  1/391]		Loss: 0.0

| Epoch [191/200] Iter [301/391]		Loss: 0.0033 Acc@1: 99.000%
| Epoch [191/200] Iter [326/391]		Loss: 0.0042 Acc@1: 99.000%
| Epoch [191/200] Iter [351/391]		Loss: 0.0021 Acc@1: 99.000%
| Epoch [191/200] Iter [376/391]		Loss: 0.0023 Acc@1: 99.000%

| Validation Epoch #191			Loss: 0.1614 Acc@1: 94.00%

=> Training Epoch #192, LR=0.0008
| Epoch [192/200] Iter [  1/391]		Loss: 0.0014 Acc@1: 100.000%
| Epoch [192/200] Iter [ 26/391]		Loss: 0.0013 Acc@1: 100.000%
| Epoch [192/200] Iter [ 51/391]		Loss: 0.0013 Acc@1: 99.000%
| Epoch [192/200] Iter [ 76/391]		Loss: 0.0019 Acc@1: 99.000%
| Epoch [192/200] Iter [101/391]		Loss: 0.0026 Acc@1: 99.000%
| Epoch [192/200] Iter [126/391]		Loss: 0.0018 Acc@1: 99.000%
| Epoch [192/200] Iter [151/391]		Loss: 0.0033 Acc@1: 99.000%
| Epoch [192/200] Iter [176/391]		Loss: 0.0022 Acc@1: 99.000%
| Epoch [192/200] Iter [201/391]		Loss: 0.0023 Acc@1: 99.000%
| Epoch [192/200] Iter [226/391]		Loss: 0.0014 Acc@1: 99.000%
| Epoch [192/200] Iter [251/391]		Loss: 0

| Epoch [199/200] Iter [101/391]		Loss: 0.0024 Acc@1: 99.000%
| Epoch [199/200] Iter [126/391]		Loss: 0.0031 Acc@1: 99.000%
| Epoch [199/200] Iter [151/391]		Loss: 0.0017 Acc@1: 99.000%
| Epoch [199/200] Iter [176/391]		Loss: 0.0031 Acc@1: 99.000%
| Epoch [199/200] Iter [201/391]		Loss: 0.0018 Acc@1: 99.000%
| Epoch [199/200] Iter [226/391]		Loss: 0.0026 Acc@1: 99.000%
| Epoch [199/200] Iter [251/391]		Loss: 0.0028 Acc@1: 99.000%
| Epoch [199/200] Iter [276/391]		Loss: 0.0021 Acc@1: 99.000%
| Epoch [199/200] Iter [301/391]		Loss: 0.0013 Acc@1: 99.000%
| Epoch [199/200] Iter [326/391]		Loss: 0.0014 Acc@1: 99.000%
| Epoch [199/200] Iter [351/391]		Loss: 0.0024 Acc@1: 99.000%
| Epoch [199/200] Iter [376/391]		Loss: 0.0014 Acc@1: 99.000%

| Validation Epoch #199			Loss: 0.1196 Acc@1: 95.00%

=> Training Epoch #200, LR=0.0008
| Epoch [200/200] Iter [  1/391]		Loss: 0.0047 Acc@1: 100.000%
| Epoch [200/200] Iter [ 26/391]		Loss: 0.0016 Acc@1: 100.000%
| Epoch [200/200] Iter [ 51/391]		Loss: 0

In [13]:
torch.save(net.state_dict(), SAVE_PATH)